## City Crime Prediction Using Clustering and Classification Techniques

Implement crime prediction final project for CS591-k1

In [1]:
import json
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import statsmodels.formula.api as smf
import pandas
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
from sklearn.cluster import KMeans
import pandas as pd

import seaborn as sns
%matplotlib inline


In [3]:
import json
from pprint import pprint

with open('boston-rows.json') as data_file:    
    bostonjson = json.load(data_file)

print len(bostonjson['data'])

columnlist = []
for col in bostonjson['meta']['view']['columns']:
    columnlist.append(col['name'])

    

268056


    Row format= 0:sequence number, 8:id, 10: type, 12:district, 13:area, 14:date, 15:weapon type, 19:year, 20:month, 21:day of week, 26:lat,long

In [4]:
print bostonjson['data'][0][0], bostonjson['data'][0][8], bostonjson['data'][0][10], bostonjson['data'][0][12], 
print bostonjson['data'][0][13], bostonjson['data'][0][14], bostonjson['data'][0][15], bostonjson['data'][0][19], 
print bostonjson['data'][0][20], bostonjson['data'][0][21], bostonjson['data'][0][27],

1 120420285 RESIDENTIAL BURGLARY D4 629 2012-07-08T06:00:00 Other 2012 7 Sunday [None, u'42.34638135', u'-71.10379454', None, False]


In [5]:
bostondf = pd.DataFrame(bostonjson['data'])
bostondf.columns = columnlist
print columnlist
print bostondf.head()[['INCIDENT_TYPE_DESCRIPTION','FROMDATE','Location']]

[u'sid', u'id', u'position', u'created_at', u'created_meta', u'updated_at', u'updated_meta', u'meta', u'COMPNOS', u'NatureCode', u'INCIDENT_TYPE_DESCRIPTION', u'MAIN_CRIMECODE', u'REPTDISTRICT', u'REPORTINGAREA', u'FROMDATE', u'WEAPONTYPE', u'Shooting', u'DOMESTIC', u'SHIFT', u'Year', u'Month', u'DAY_WEEK', u'UCRPART', u'X', u'Y', u'STREETNAME', u'XSTREETNAME', u'Location']
  INCIDENT_TYPE_DESCRIPTION             FROMDATE  \
0      RESIDENTIAL BURGLARY  2012-07-08T06:00:00   
1        AGGRAVATED ASSAULT  2012-07-08T06:03:00   
2                   ROBBERY  2012-07-08T06:26:00   
3       COMMERCIAL BURGLARY  2012-07-08T06:56:00   
4                   ROBBERY  2012-07-08T07:15:00   

                                         Location  
0  [None, 42.34638135, -71.10379454, None, False]  
1  [None, 42.31684135, -71.07458456, None, False]  
2  [None, 42.34284135, -71.09698955, None, False]  
3   [None, 42.3164411, -71.06582908, None, False]  
4  [None, 42.27051636, -71.11989955, None, False] 

In [24]:
print bostonjson['data'][0][12]

D4


In [25]:
print bostondf['Location'][0][1]
latlist = map(lambda k: k[1], bostondf['Location'])
longlist = map(lambda k: k[2], bostondf['Location'])

typelist = map(lambda k: k, bostondf['INCIDENT_TYPE_DESCRIPTION'])
hourlist = map(lambda k: k.split("T")[1].split(":")[0], bostondf['FROMDATE'])
daylist = map(lambda k: k, bostondf['DAY_WEEK'])

print len(latlist), len(longlist), len(hourlist)

42.34638135
268056 268056 268056


In [26]:
areacodelist = map(lambda k: k[12], bostonjson['data'])
print len(areacodelist)

268056


In [27]:
streetlist = map(lambda k: k, bostondf['STREETNAME'])

In [28]:
asdata={"Area_Code":areacodelist,"Street_Name":streetlist}
asdf=pd.DataFrame(asdata)

In [29]:
streets=asdf.groupby(['Area_Code'])

In [30]:
for a,b in streets:
    if a=='HTU':
        print b

       Area_Code      Street_Name
143983       HTU       PARK PLAZA
173882       HTU       BRAGDON ST
178159       HTU        STUART ST
234844       HTU    WASHINGTON ST
248544       HTU  COMMONWEALTH AV
249846       HTU  COMMONWEALTH AV
249907       HTU  COMMONWEALTH AV
253200       HTU  COMMONWEALTH AV


In [31]:
print len(streetlist)

268056


In [32]:
import pandas as pd
import warnings

Assaultlist=[]
DrugAlcohollist=[]
Othercrimeslist=[]
PublicDisorderlist=[]
Theftlist=[]
WhiteCollarCrimelist=[]
BurglaryCrimelist=[]
VandalismCrimelist=[]
ArrestCrimelist=[]
LarcenyCrimelist=[]
HarassmentCrimelist=[]
SexCrimelist=[]
FireCrimelist=[]
T1list=[]
T2list=[]
T3list=[]
T4list=[]
T5list=[]
T6list=[]
Weekdaylist=[]
Weekendlist=[]

crimedata ={"AreaCode":areacodelist,"Crime_type":typelist,"Hour":hourlist,"Day":daylist}


In [33]:
crimedf=pd.DataFrame(crimedata)

In [34]:
print len(crimedf)

268056


In [35]:
crimedf=crimedf.loc[crimedf['AreaCode']!='NULL']
crimedf=crimedf.loc[crimedf['Crime_type'] !=('BENoProp')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('PropFound')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('InvProp')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('PersLoc')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('MVAcc')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Ballist')]

crimedf=crimedf.loc[crimedf['Crime_type'] !=('Argue')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Harbor')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Restrain')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Gather')]


crimedf=crimedf.loc[crimedf['Crime_type'] !=('Service')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('SkipFare')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('SearchWarr')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('PersMiss')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('DEATH INVESTIGATION')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('LICViol')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Landlord')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Hazardous')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Plates')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('07RV')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('PropDam')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('HOMICIDE')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('DISORDERLY')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('HateCrim')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('TRESPASS')]

crimedf=crimedf.loc[crimedf['Crime_type'] !=('ARSON')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('BurgTools')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Explos')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('InvVeh')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Bomb')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Aircraft')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Labor')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Manslaug')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('BioThreat')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Property Found')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Counterfeiting')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Verbal Disputes')]

crimedf=crimedf.loc[crimedf['Crime_type'] !=('License Plate Related Incidents')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Motor Vehicle Accident Response')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Police Service Incidents')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Investigate Person')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Property Lost')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Towed')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Ballistics')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Disorderly Conduct')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Violations')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Recovered Stolen Property')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Assembly or Gathering Violations')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Confidence Games')]

crimedf=crimedf.loc[crimedf['Crime_type'] !=('Investigate Property')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Missing Person Located')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Auto Theft Recovery')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Property Related Damage')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Search Warrants')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Evading Fare')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Bomb Hoax')]
crimedf=crimedf.loc[crimedf['Crime_type'] !=('Homicide')]









In [36]:
print len(crimedf)
crimedf=crimedf.reset_index()


209515


In [37]:
print crimedf.head(n=5)

   index AreaCode            Crime_type     Day Hour
0      0       D4  RESIDENTIAL BURGLARY  Sunday   06
1      1       B2    AGGRAVATED ASSAULT  Sunday   06
2      2       D4               ROBBERY  Sunday   06
3      3       B2   COMMERCIAL BURGLARY  Sunday   06
4      4      E18               ROBBERY  Sunday   07


In [38]:
del crimedf['index']

In [39]:
for index, row in crimedf.iterrows():
    if row['Crime_type'] in ('AGGRAVATED ASSAULT','SIMPLE ASSAULT','Simple Assault','Aggravated Assault','Indecent Assault'):
        Assaultlist.append(1)
    else:
        Assaultlist.append(0)
    if row['Crime_type'] in ('DRUG CHARGES','VIOLATION OF LIQUOR LAWS','Operating Under the Influence','OPERATING UNDER INFLUENCE','Drug Violation','PubDrink','Liquor Violation'):
        DrugAlcohollist.append(1)
    else:
        DrugAlcohollist.append(0)
    if row['Crime_type'] in ('RESIDENTIAL BURGLARY','COMMERCIAL BURGLARY','Commercial Burglary','Other Burglary','Residential Burglary'):
        BurglaryCrimelist.append(1)
    else:
        BurglaryCrimelist.append(0)
    
    if row['Crime_type'] in ('DISORDERLY','PubDrink'):
        PublicDisorderlist.append(1)
    else:
        PublicDisorderlist.append(0)
    if row['Crime_type'] in ('AUTO THEFT','STOLEN PROPERTY CHARGES','ROBBERY','Auto Theft','Auto Theft Recovery','STOLEN PROPERTY CHARGES','Robbery'):
        Theftlist.append(1)
    else:
        Theftlist.append(0)
    if row['Crime_type'] in ('FRAUD','EMBEZELLMENT','Fraud','Embezzlement'):
        WhiteCollarCrimelist.append(1)
    else:
        WhiteCollarCrimelist.append(0)
    if row['Crime_type'] in ('Vandalism','VANDALISM'):
        VandalismCrimelist.append(1)
    else:
        VandalismCrimelist.append(0)
    if row['Crime_type'] in ('Arrest','Warrant Arrests','Prisoner Related Incidents'):
        ArrestCrimelist.append(1)
    else:
        ArrestCrimelist.append(0)
    if row['Crime_type'] in ('OTHER LARCENY','LARCENY FROM MOTOR VEHICLE','Larceny','Larceny From Motor Vehicle'):
        LarcenyCrimelist.append(1)
    else:
        LarcenyCrimelist.append(0)
    if row['Crime_type'] in ('Harass','Criminal Harassment'):
        HarassmentCrimelist.append(1)
    else:
        HarassmentCrimelist.append(0)
    if row['Crime_type'] in ('FIRE','Fire Related Reports','Firearm Discovery','Explosives','Firearm Violations'):
        FireCrimelist.append(1)
    else:
        FireCrimelist.append(0)
    if row['Crime_type'] in ('SexReg','Sex Offender Registration','Prostitution','PROSTITUTION CHARGES','Rape and Attempted'):
        SexCrimelist.append(1)
    else:
        SexCrimelist.append(0)
    if row['Crime_type'] in ('VAL','MedAssist','Harbor Related Incidents','Arson','License Violation','Landlord/Tenant Disputes','Restraining Order Violations','Offenses Against Child/Family','Missing Person Reported','Medical Assistance','Counterfeiting','Runaway','GAMBLING OFFENSE','FORGERY','InvPer','TOWED','WEAPONS CHARGE','PropLost','CRIMES AGAINST CHILDREN','PhoneCalls','OTHER','Other','PRISON'):
        Othercrimeslist.append(1)
    else:
        Othercrimeslist.append(0)
    if row['Hour'] in ('01','02','03','04'):
        T1list.append(1)
    else:
        T1list.append(0)
    if row['Hour'] in ('05','06','07','08'):
        T2list.append(1)
    else:
        T2list.append(0)
    if row['Hour'] in ('09','10','11','12'):
        T3list.append(1)
    else:
        T3list.append(0)
    if row['Hour'] in ('13','14','15','16'):
        T4list.append(1)
    else:
        T4list.append(0)
    if row['Hour'] in ('17','18','19','20'):
        T5list.append(1)
    else:
        T5list.append(0)
    if row['Hour'] in ('21','22','23','00'):
        T6list.append(1)
    else:
        T6list.append(0)
    if row['Day'] in ('Monday','Tuesday','Wednesday','Thursday'):
        Weekdaylist.append(1)
    else:
        Weekdaylist.append(0)
    if row['Day'] in ('Friday','Saturday','Sunday'):
        Weekendlist.append(1)
    else:
        Weekendlist.append(0)

In [40]:

        
crime_typedata={"Assault":Assaultlist,"DrugAlcohol":DrugAlcohollist,"PublicDisorder":PublicDisorderlist,
               "Theft":Theftlist,"WhiteCollarCrime":WhiteCollarCrimelist,"Othercrimes":Othercrimeslist,"Burglary":BurglaryCrimelist,
               "Vandalism":VandalismCrimelist,"Arrest":ArrestCrimelist,"Larcency":LarcenyCrimelist,
               "Harassment":HarassmentCrimelist,"Sex":SexCrimelist,"Fire":FireCrimelist}


crime_typedf=pd.DataFrame(crime_typedata)








   
   
    
        
        
        





In [41]:




Timedata={"T1":T1list,"T2":T2list,"T3":T3list,"T4":T4list,"T5":T5list,"T6":T6list,
         "Weekday":Weekdaylist,"Weekend":Weekendlist}

timedf=pd.DataFrame(Timedata)

pd.options.mode.chained_assignment = None
#print crimedf.Day.unique()

df=crimedf[['AreaCode']]



In [42]:
df1 = pd.concat([df, crime_typedf], axis=1)
Crimematrix = pd.concat([df1, timedf], axis=1)



In [43]:
Crimematrix1= Crimematrix .groupby(['AreaCode'])

In [44]:
print Crimematrix.columns.values

['AreaCode' 'Arrest' 'Assault' 'Burglary' 'DrugAlcohol' 'Fire' 'Harassment'
 'Larcency' 'Othercrimes' 'PublicDisorder' 'Sex' 'Theft' 'Vandalism'
 'WhiteCollarCrime' 'T1' 'T2' 'T3' 'T4' 'T5' 'T6' 'Weekday' 'Weekend']


In [45]:
Area=[]
TotalArrest=[]
TotalAssault=[]
TotalBurglary=[]
TotalDrugAlcohol=[]
TotalFire=[]
TotalHarassment=[]
TotalLarcency=[]
TotalOthercrimes=[]
TotalPublicDisorder=[]
TotalSex=[]
TotalTheft=[]
TotalVandalism=[]
TotalWhiteCollarCrime=[]
TotalT1=[]
TotalT2=[]
TotalT3=[]
TotalT4=[]
TotalT5=[]
TotalT6=[]
TotalWeekday=[]
TotalWeekend=[]



In [46]:
for a,b in Crimematrix1:
    Area.append(a)
    TotalArrest.append(b['Arrest'].sum())
    TotalAssault.append(b['Assault'].sum())
    TotalBurglary.append(b['Burglary'].sum())
    TotalDrugAlcohol.append(b['DrugAlcohol'].sum())
    TotalFire.append(b['Fire'].sum())
    TotalHarassment.append(b['Harassment'].sum())
    TotalLarcency.append(b['Larcency'].sum())
    TotalOthercrimes.append(b['Othercrimes'].sum())
    TotalPublicDisorder.append(b['PublicDisorder'].sum())
    TotalSex.append(b['Sex'].sum())
    TotalTheft.append(b['Theft'].sum())
    TotalVandalism.append(b['Vandalism'].sum())
    TotalWhiteCollarCrime.append(b['WhiteCollarCrime'].sum())
    TotalT1.append(b['T1'].sum())
    TotalT2.append(b['T2'].sum())
    TotalT3.append(b['T3'].sum())
    TotalT4.append(b['T4'].sum())
    TotalT5.append(b['T5'].sum())
    TotalT6.append(b['T6'].sum())
    TotalWeekday.append(b['Weekday'].sum())
    TotalWeekend.append(b['Weekend'].sum())
    
    

In [47]:
lregdata={"AreaCode":Area,"Arrest":TotalArrest,"Assault":TotalAssault,"Burglary":TotalBurglary,
          "DrugAlochol":TotalDrugAlcohol,"Fire":TotalFire,"Harassment":TotalHarassment,
          "Larcency":TotalLarcency,"PublicDisorder":TotalPublicDisorder,
          "Sex":TotalSex,"Theft":TotalTheft,"Vandalism":TotalVandalism,"WhiteCollar":TotalWhiteCollarCrime,
          "T1":TotalT1,"T2":TotalT2,"T3":TotalT3,"T4":TotalT4,"T5":TotalT5,"T6":TotalT6,
          "Weekday":TotalWeekday,"Weekend":TotalWeekend,"Other":TotalOthercrimes}
          

In [48]:
linerregdf=pd.DataFrame(lregdata)

In [49]:
linerregdf['TotalCrimes']=linerregdf['Arrest']+linerregdf['Assault']+linerregdf['Burglary']+linerregdf['DrugAlochol']+linerregdf['Fire']+linerregdf['Harassment']+linerregdf['Larcency']+linerregdf['PublicDisorder']+linerregdf['Sex']+linerregdf['Theft']+linerregdf['Vandalism']+linerregdf['WhiteCollar']+linerregdf['T1']+linerregdf['T2']+linerregdf['T3']+linerregdf['T4']+linerregdf['T5']+linerregdf['T6']+linerregdf['Weekday']+linerregdf['Weekend']+linerregdf['Other']

In [50]:
print linerregdf

   AreaCode  Arrest  Assault  Burglary  DrugAlochol  Fire  Harassment  \
0        A1     343     3086       515         1371   114          41   
1       A15      32      471       130          195    47          10   
2        A7     120     1257       438          961   133          26   
3        B2     359     4126      1197         3207   303          37   
4        B3     300     2808       830         1272   237          15   
5       C11     314     3473      1125         2005   274          53   
6        C6     167     1610       544         1636   150          26   
7       D14      64     1454      1108          577   158          48   
8        D4     184     3062      1107         1383   199          67   
9       E13     106     1139       736          786   119          19   
10      E18      76     1043       493          825   161          20   
11       E5      59      817       467          784   123          31   
12      HTU       0        0         0            0

In [51]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

X=linerregdf[['Arrest','Assault','Burglary','DrugAlochol','Harassment','Larcency','PublicDisorder','Sex','Theft','Vandalism','WhiteCollar','Other']]



Y=linerregdf[['TotalCrimes']]

print X.shape
print Y.shape



ols_model = sm.OLS(Y, X)
ols_results = ols_model.fit()

print(ols_results.summary())

(13, 12)
(13, 1)
                            OLS Regression Results                            
Dep. Variable:            TotalCrimes   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.158e+09
Date:                Mon, 25 Apr 2016   Prob (F-statistic):           1.09e-05
Time:                        01:05:21   Log-Likelihood:                0.94924
No. Observations:                  13   AIC:                             22.10
Df Residuals:                       1   BIC:                             28.88
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Arrest             4.2771  

C:\Users\VINITKUMARGOUD\Anaconda2\lib\site-packages\scipy\stats\stats.py:1277: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


In [52]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


X=linerregdf[['T1','T2','T3','T4','T5','T6']]
Y=linerregdf[['TotalCrimes']]

print X.shape
print Y.shape



ols_model = sm.OLS(Y, X)
ols_results = ols_model.fit()

print(ols_results.summary())



(13, 6)
(13, 1)
                            OLS Regression Results                            
Dep. Variable:            TotalCrimes   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 6.676e+05
Date:                Mon, 25 Apr 2016   Prob (F-statistic):           8.73e-20
Time:                        01:05:22   Log-Likelihood:                -74.395
No. Observations:                  13   AIC:                             160.8
Df Residuals:                       7   BIC:                             164.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
T1             3.1397      0.103    

In [53]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


X=linerregdf[['Weekday','Weekend']]
Y=linerregdf[['TotalCrimes']]

print X.shape
print Y.shape



ols_model = sm.OLS(Y, X)
ols_results = ols_model.fit()

print(ols_results.summary())


(13, 2)
(13, 1)
                            OLS Regression Results                            
Dep. Variable:            TotalCrimes   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 9.627e+05
Date:                Mon, 25 Apr 2016   Prob (F-statistic):           1.46e-29
Time:                        01:05:23   Log-Likelihood:                -82.095
No. Observations:                  13   AIC:                             168.2
Df Residuals:                      11   BIC:                             169.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Weekday        3.0382      0.047    

In [61]:

logregdf=linerregdf
logregdf['SafePercentage']=100-((logregdf['TotalCrimes']/logregdf['TotalCrimes'].sum())*100)

In [65]:
def Exc(row):
    if row['SafePercentage']>95:
        return 1
    else:
        return 0

In [66]:
logregdf['Safe_Area'] = logregdf.apply (lambda row: Exc (row),axis=1)

In [75]:
print logregdf

   AreaCode  Arrest  Assault  Burglary  DrugAlochol  Fire  Harassment  \
0        A1     343     3086       515         1371   114          41   
1       A15      32      471       130          195    47          10   
2        A7     120     1257       438          961   133          26   
3        B2     359     4126      1197         3207   303          37   
4        B3     300     2808       830         1272   237          15   
5       C11     314     3473      1125         2005   274          53   
6        C6     167     1610       544         1636   150          26   
7       D14      64     1454      1108          577   158          48   
8        D4     184     3062      1107         1383   199          67   
9       E13     106     1139       736          786   119          19   
10      E18      76     1043       493          825   161          20   
11       E5      59      817       467          784   123          31   
12      HTU       0        0         0            0

In [76]:
res=logregdf[['AreaCode','SafePercentage','Safe_Area']]
res

,AreaCode,SafePercentage,Safe_Area
0,A1,88.800052,0
1,A15,97.901372,1
2,A7,95.132359,1
3,B2,85.222284,0
4,B3,91.269420,0
5,C11,87.115265,0
6,C6,92.166446,0
7,D14,92.661242,0
8,D4,85.145783,0
9,E13,94.365433,0


In [58]:

X1=logregdf[['Arrest','Harassment','Theft','WhiteCollar','Larcency','Vandalism']]



Y1=logregdf[['Safe_Area']]


logit = sm.Logit(Y1,X1)
result = logit.fit() 

print result.summary()

Optimization terminated successfully.
         Current function value: 0.250140
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:              Safe_Area   No. Observations:                   13
Model:                          Logit   Df Residuals:                        7
Method:                           MLE   Df Model:                            5
Date:                Mon, 25 Apr 2016   Pseudo R-squ.:                  0.5947
Time:                        01:05:27   Log-Likelihood:                -3.2518
converged:                       True   LL-Null:                       -8.0241
                                        LLR p-value:                   0.08921
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Arrest          0.1016      0.140      0.726      0.468        -0.173     0.376
Harassment      0.6262   

In [59]:

X1=logregdf[['T1','T2','T3','T4','T5','T6']]


Y1=logregdf[['Safe_Area']]


logit = sm.Logit(Y1,X1)
result = logit.fit() 

print result.summary()

Optimization terminated successfully.
         Current function value: 0.066764
         Iterations 14
                           Logit Regression Results                           
Dep. Variable:              Safe_Area   No. Observations:                   13
Model:                          Logit   Df Residuals:                        7
Method:                           MLE   Df Model:                            5
Date:                Mon, 25 Apr 2016   Pseudo R-squ.:                  0.8918
Time:                        01:05:27   Log-Likelihood:               -0.86793
converged:                       True   LL-Null:                       -8.0241
                                        LLR p-value:                   0.01374
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
T1            -0.0717      0.251     -0.285      0.775        -0.564     0.421
T2            -0.1182      0

In [60]:
X1=logregdf[['Assault','Burglary','DrugAlochol','PublicDisorder','Sex','Other']]



Y1=logregdf[['Safe_Area']]


logit = sm.Logit(Y1,X1)
result = logit.fit() 

print result.summary()

Optimization terminated successfully.
         Current function value: 0.208583
         Iterations 14
                           Logit Regression Results                           
Dep. Variable:              Safe_Area   No. Observations:                   13
Model:                          Logit   Df Residuals:                        7
Method:                           MLE   Df Model:                            5
Date:                Mon, 25 Apr 2016   Pseudo R-squ.:                  0.6621
Time:                        01:05:28   Log-Likelihood:                -2.7116
converged:                       True   LL-Null:                       -8.0241
                                        LLR p-value:                   0.05934
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Assault           -0.0007      0.011     -0.068      0.945        -0.021     0.020
Burglary        